In [1]:
import pandas as pd
from aw_client import Session

token = open('token.txt').read()
session = Session(token=token, aw_url='https://aw-demo.ru')

df = session.load_model(model_id=13839)
print(df.shape)
df.head(3)

(482937, 23)


,id,donor_id,purpose_id,transaction_id,recurring_id,status,when,amount,amount_total,category,...,default_69s4__id,sex,default_fvyq__id,default_fvyq__when,default_fvyq__amount,currency,ipcountry,ipregion,default_h5ib__id,fund_id
0,79b04f84bde1b0f6a681902590c0907d,18dd5d0a449d7db07c2a9ec0e896b193,6fde334fe1ae0f7b04556a2f1395088d,4644dc9cf783db3f916aec717c295450,c443d9e0d54b519a8bfc6baa51c6148b,3,2017-08-31 16:19:00,100,100,card,...,18dd5d0a449d7db07c2a9ec0e896b193,,4644dc9cf783db3f916aec717c295450,2017-08-29 15:14:18,100,RUB,,,6fde334fe1ae0f7b04556a2f1395088d,030411b7e76338a91180dec47a354182
1,85d723af187b59b189c160a27fd0fd06,5c0311678207b949070c370d1fd956d3,b0cecf1a7fecaad80a37bac7198d9515,56d309cf2d6d8d599e1d573ef9758fbf,33038f6ab8efaad0a62ddccbea686e66,3,2020-12-08 13:28:00,1000,1000,card,...,5c0311678207b949070c370d1fd956d3,f,56d309cf2d6d8d599e1d573ef9758fbf,2020-12-08 10:28:38,1000,RUB,,,b0cecf1a7fecaad80a37bac7198d9515,e3fd213a01aee164f475152137a83eec
2,94ae7018c15c829bc5edb93756dcb25d,966a63f7d5cdd1a8e203cf3325b979b8,b9a7e4b991ffc87c72773aa103f5b963,4e6a390dc0c45f8d3b45c833dbca29d5,,5,2021-06-27 13:33:06,"257,74",280,sms,...,966a63f7d5cdd1a8e203cf3325b979b8,,4e6a390dc0c45f8d3b45c833dbca29d5,2021-06-27 13:33:04,280,RUB,Россия,Астраханская обл.,b9a7e4b991ffc87c72773aa103f5b963,e3fd213a01aee164f475152137a83eec


In [2]:
# убираем дубликаты транзакций
df.drop_duplicates(subset=['transaction_id'], inplace=True) 

# убираем ненужные id после джойнов
df.drop(['default_69s4__id', 'default_fvyq__id', 'default_h5ib__id', 'purpose_id'], axis=1, inplace=True)

# убираем донора со странными статусами (выброс или плохо заполнили про него бд)
df = df[df['donor_id']!='8a658517fd414044e03969015b7d641b'] 

# восстаналиваем ранее найденные статусы
df.loc[df['id']=='d1e178e8afe3f0768984985366c07c54', 'status']=3
df.loc[df['id']=='9010d34690130db12781a5f7ac194fe7', 'status']=3 

# убираем не конечные статусы
df = df[~df.status.isin([6, 2, 1, -1])] 

In [3]:
# после очистки от ненужных транзакций создаем новый датасет с донорами, объединив всех доноров из таблицы Пожертвования
donor_df = pd.DataFrame(df['donor_id'].unique(), columns=['id']) 

# создадим ранее полученный список доноров, которые возвращали все свои пожертвования
returned_donation = ['1cb4c44c0d1b883e1771376debf87b4f', '1d1fb540c4c118b0acd2bd200a403436', '21d8377d0c0d4bc305645fefba6a46e1',
                     '525ceba5dbd514d2a83bbda53331a4a9', '52b081c8eb41656b6cb56530c2f2df1a', '61887e9da9a4cb56535b0377e28abb35',
                     '65ab81e7dd3df787e10bd337b4e6710c', '78650a5f5d96b42588163993b6201bee', '977731a963b96ab302a8c6a36897924a',
                     'b32cad5aea8f8bfb7904bda7aef54121', 'bbb6856a9de00231c01ddca80fee5f32', 'c0429bef575b15c2b29c042adadc1db4',
                     'cd173ddf0f1e20a7d077b91b4e78b776', 'f71d23e76db89728b6a65f4eb80f745e', 'fcda9e7d486a2505bd26e0af5a9c1efb']

# добавляем метку для тех, кто возвращал все свои пожертвования
donor_df['only_returned_donation'] = donor_df['id'].isin(returned_donation).astype('int') 


In [4]:
# удаляем ненужные колонки после джойнов
df.drop(['default_fvyq__when', 'default_fvyq__amount'], inplace=True, axis=1)

# создадим корректный тип данных для времени
df['when'] = pd.to_datetime(df['when'])

# поправим тип данных на числовой
df['amount_total'] = df['amount_total'].astype(str).str.replace(',', '.').astype(float) 

# удаляем выявленные после анализа ненужные колонки
df.drop(['amount', 'device_type', 'os', 'currency', 'sex'], inplace=True, axis=1)

In [5]:
# в таблице доноров классифицируем их на активных и неактивных

# создадим дату для отсчета, так как данные только до 2021 года, то просто отступим от последней даты из датасета на один день
current_date = pd.Timestamp('2021-07-01 23:59:59')

# найдем для каждого донора последнюю дату его пожертвования
last_complited_donation_date = df[df.status==3].groupby('donor_id')['when'].max().reset_index()
last_complited_donation_date.rename(columns={'when':'last_date'}, inplace=True)

# подсоединим эти данные к донорам 
donor_df = donor_df.merge(last_complited_donation_date, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', axis=1, inplace=True)

# там где значение даты не пропущено подсчитаем количество дней от последнего пожертвования до текущей даты
donor_df.loc[donor_df['last_date'].notnull(), 'days_since_last'] = (current_date - donor_df['last_date']).dt.days

# пропуски у доноров, не имеющих транзакций со статусом 3 (например у тех, чьи пожертвования отклонил банк) 
# заполним максимальным значением подсчитанных days_since_last 
donor_df['days_since_last'].fillna(donor_df['days_since_last'].max(), inplace=True)
donor_df['last_date'].fillna(0, inplace=True)

# поставим метку active для активных доноров(менее 200 дней после последнего пожертвования) равным 1 иначе 0
donor_df['active'] = (donor_df['days_since_last'] <= 200).astype('int')

In [6]:
# перейдем к пожертвованиям
# восстаналиваем пропуски в способе оплаты
df.loc[df['category']=="", 'category'] = df['gateway']
#.fillna(df['gateway'], inplace=True)

# далее объединим данные столбцы про оплату в единый призак, так как часто значения в них дублируют друг друга

# получим более подробную информацию для общих способов оплаты
change = ['sms','card','bankcard','sbp','Other','paypal_all','bank_card','all','BankCard','mobile','text_box','bank_order']
for value in change:
    df.loc[df['category'] == value, 'category'] = df['gateway']
    
# после анализа объединим редкие способы оплаты и платежные системы в группы в соответствии со словарями
gateway_dict = {'other': ['dolyame', 'quittance', 'text', 'sovcombank']}

category_dict = {'other': ['dolyame', 'quittance', 'text', 'sovcombank'], 'robokassa': ['WMR'], 'mir': ['mir_pay'],
                 'yandex': ['yandex_all', 'yandex_ab', 'yoo_money', 'yandex_sb', 'yandex_card']}

# напишем функцию для замены
def replace_value(value, column):
    """
    value: str - текущее значение категории
    column: str - название колонки для выбора словаря
    return value: str - новое значение категории
    """
    if column  == 'gateway':
        for key, values in gateway_dict.items():
            if value in values:
                return key
    else:
        for key, values in category_dict.items():
            if value in values:
                return key
    return value  # возвращаем оригинальное значение, если не найдено соответствие

# применим функцию к колонкам
df['gateway'] = df['gateway'].agg(lambda x: replace_value(x,'gateway'))
df['category'] = df['category'].agg(lambda x: replace_value(x,'category'))

# заменим способы оплаты yandex_money, yandex_pay, Qiwi30Ocean на соответствующую платежную систему для более подробной информации
df.loc[df['category'] == 'yandex_money', 'category'] = df['gateway']
df.loc[df['category'] == 'yandex_pay', 'category'] = df['gateway']
df.loc[df['category'] == 'Qiwi30Ocean', 'category'] = df['gateway']

# заменим платежные системы mixplat,cloudpayments,robokassa,yandex на соответствующие типы оплаты для более подробной информации
for value in ['mixplat', 'cloudpayments', 'robokassa', 'yandex']:
    df.loc[df['gateway'] == value, 'gateway'] = df['category']  
    
# объединим значения сбербанка
df.loc[df['gateway'] == 'sber', 'gateway'] = 'sberbank'
df.loc[df['gateway'] == 'sberpay', 'gateway'] = 'sberbank'

# уберем ненужный признак, сохранив всю информацию об оплате в одной колонке gateway
df.drop('category', inplace=True, axis=1)

In [7]:
# из таблицы с пожертвованиями выделим основную плтаежную систему для каждого донора
main_gateway = df.groupby('donor_id')['gateway'].agg(lambda x: x.mode().iloc[0]).reset_index()
donor_df = donor_df.merge(main_gateway, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', inplace=True, axis=1)

In [8]:
# для стран заменим международные коды на понятные названия
words = {'RU': 'Россия', 'ES': 'Испания', 'GB': 'Великобритания', 'US': 'США', 'DE': 'Германия', 'UA': 'Украина', 'LK': 'США',
         'KR': 'Республика Корея', 'FR': 'Франция', 'AE': 'ОАЭ', 'AT': 'Австрия', 'IL': 'Израиль', 'LT': 'Литва', 'CY': 'Кипр',
         'NO': 'Норвегия', 'IT': 'Италия', 'CH': 'Швейцария', 'EE': 'Эстония', 'KZ': 'Казахстан', 'MT': 'Мальта', 'IN': 'Индия',
         'MD': 'Молдавия', 'TR': 'Турция', 'PL': 'Польша', 'SG': 'Сингапур', 'NL': 'Нидерланды', 'LV': 'Латвия', 'SE': 'Швеция',
         'BE': 'Бельгия', 'BY': 'Белоруссия', 'IE': 'Ирландия', 'HR': 'Хорватия', 'GR': 'Греция', 'AM': 'Армения', 'PR': 'США',
         'HU': 'Венгрия', 'CN': 'КНР', 'CA': 'Канада', 'ME': 'Черногория', 'CZ': 'Чехия', 'DK': 'Дания', 'PT': 'Португалия',
         'FI': 'Финляндия', 'BG': 'Болгария', 'VN': 'Вьетнам', 'MU': 'Маврикий', 'SI': 'Словения', 'GE': 'Грузия', 'IQ': 'Ирак',
         'TH': 'Таиланд', 'AR': 'Аргентина', 'RO': 'Румыния', 'JP': 'Япония', 'EG': 'Египет', 'AU': 'Австралия', 'OM': 'Оман', 
         'DO': 'Чили', 'CU': 'Куба', 'NG': 'Нигерия', 'MX': 'Мексика', 'CR': 'Коста-Рика', 'BA': 'Федерация Боснии и Герцеговины',
         'ID': 'Индонезия', 'VG': 'Великобритания', 'MK': 'Македония', 'MY': 'Малайзия', 'KG': 'Киргизия', 'LU': 'Люксембург',
         'BR': 'Бразилия', 'LB': 'Ливан', 'KH': 'Камбоджа', 'UZ': 'Узбекистан', 'SC': 'Канада', 'ZA': 'ЮАР', 'AZ': 'Азербайджан',
         'RS': 'Сербия', 'DZ': 'Африка', 'SA': 'Саудовская Аравия', 'TW': 'Китай', 'QA': 'Катар', 'MV': 'Мальдивы', 'IR': 'Иран',
         'EC': 'Эквадор', 'NZ': 'Новая Зеландия', 'TJ': 'Таджикистан', 'PK': 'Пакистан', 'VE': 'Венесуэла', 'BD': 'Бангладеш', 
         'JM': 'Ямайка', 'CL': 'Чили', 'KN': 'Сент-Китс и Невис',  'HK': 'Китай', 'MO': 'Китай', 'MF': 'Франция','AD': 'Андорра',
         'NP': 'Непал', 'SK': 'Словакия', 'IS': 'Исландия', 'AL': 'Албания', 'CW': 'Нидерланды', 'MC': 'Монако', 'UY': 'Уругвай',
         'KW': 'Кувейт', 'MA': 'Марокко', 'JO': 'Иордания', 'KY': 'Великобритания', 'TM': 'Туркменистан', 'ндия Индия': 'Индия',
         'Макао': 'Китай', 'Гонконг': 'Китай', "": None}

df['ipcountry'].replace(words, inplace=True)

In [9]:
# пропущенные регионы в транзакциях заменим соответствующей страной

df['ipregion'].replace({"": None}, inplace=True)
df['ipregion'].fillna(df['ipcountry'], inplace=True)

In [10]:
# восстановим оставшиеся пропуски в регионах/странах, заполнив их самым частым регионом/страной текущего донора
# получим моду каждого донора для стран и регионов
temp_country = df.groupby('donor_id')['ipcountry'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index()
temp_country.rename(columns={'ipcountry': 'country'}, inplace=True)

temp_region = df.groupby('donor_id')['ipregion'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index()
temp_region.rename(columns={'ipregion': 'region'}, inplace=True)

# запишем для транзакций значения этих мод
df = df.merge(temp_country, how='left', on='donor_id')
df = df.merge(temp_region, how='left', on='donor_id')

# с помощью получившихся колонок заполним пропуски
df['ipcountry'].fillna(df['country'], inplace=True)
df['ipregion'].fillna(df['region'], inplace=True)

# удалим лишние колонки
df.drop(['country', 'region'], inplace=True, axis=1)

In [11]:
# оставшиеся пропуски отнесем к неизвестной категории
df['ipcountry'].fillna('Неизвестно', inplace=True)
df['ipregion'].fillna('Неизвестно', inplace=True)

In [12]:
# теперь после заполнения всех пропусков можно создать для доноров признак с их основной страной, напишем функцию
# если у донора несколько мод и одна из них относится к неизвестной категории, то мы возвращем другую
# например у донора было одинаково много пожертвований из Франции и из страны Неизвестно, оставим модой только значение Франция
def get_true_country_mode(value):
    """
    value: list - список самых частых стран донора (мода)
    return value: list - новый список без значения 'Неизвестно', если оно не единственное 
    """
    if 'Неизвестно' in value:
        value.remove('Неизвестно')
        return ['Неизвестно'] if not value else value
    else:
        return value
    
# применим данную функцию
temp_country_mode = df.groupby('donor_id')['ipcountry'].agg(lambda x: get_true_country_mode(x.mode().tolist())).reset_index()

# поставим донорам, у которых часто транзакции не только из одной страны специальную метку
def get_multiple(value):
    """
    value: list - список самых частых стран донора (мода)
    return value: int - 1 если мод несколько, 0 если мода одна
    """
    if len(value)>1:
        return 1
    return 0

temp_country_mode['multiple_country'] = temp_country_mode['ipcountry'].apply(get_multiple)

# поставим донорам, которые часто жертвовали из России специальную метку
def get_russia(value):
    """
    value: list - список самых частых стран донора (мода)
    return value: int - 1 если среди мод есть Россия, 0 если нет
    """
    if 'Россия' in value:
        return 1
    return 0

temp_country_mode['russia'] = temp_country_mode['ipcountry'].apply(get_russia)

In [13]:
# проверим все ли верно 

temp_country_mode[temp_country_mode.multiple_country==1].head()

,donor_id,ipcountry,multiple_country,russia
368,0070edebbba4542670916eab24d0beb1,"[Россия, США]",1,1
1070,013b941fa727773dc62f3b757744a1b0,"[Россия, Эстония]",1,1
1252,016d01e3b4671702973c350d628b7b3b,"[Германия, Россия]",1,1
1727,01f202e950d76be1c8a052320cfeca13,"[Италия, Франция]",1,0
2557,030befda3225f02b777c7c3114c43142,"[Россия, Франция]",1,1


In [14]:
# добавим ранее созданные метки из датасета с модой признака ipcountry
# то есть добавим метку multiple_country и russia
donor_df = donor_df.merge(temp_country_mode, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', axis=1, inplace=True)

# удалим признак 'ipcountry', так как в нем сейчас находятся несколько мод, а мы потом добавим одну
donor_df.drop(['ipcountry'], inplace=True, axis=1)

In [15]:
# перед заполнением информации непосредственно о стране донора, а не о моде признака ipcountry, объединим редкие страны в группы 
# создадим словарь с объединением стран 
country_dict = {'Европа': ['Эстония', 'Швейцария', 'Латвия', 'Монако', 'Швеция', 'Федерация Боснии и Герцеговины',
                           'Чехия', 'Греция', 'Румыния', 'Норвегия', 'Австрия', 'Кипр', 'Сербия', 'Люксембург', 
                           'Бельгия', 'Португалия', 'Черногория', 'Венгрия', 'Ирландия', 'Литва', 'Дания', 'Хорватия',
                           'Польша', 'Словения', 'Мальта', 'Албания', 'Македония', 'Исландия', 'Болгария', 'Словакия', 
                           'Андорра', 'Новая Зеландия', 'Австралия', 'Италия', 'Испания', 'Финляндия'],
                'Азия': ['Турция', 'Израиль', 'Бангладеш', 'Грузия', 'Индия', 'Таиланд', 'Сингапур', 'Египет', 'Оман',
                         'Республика Корея', 'Камбоджа', 'Малайзия', 'Иран', 'Иордания', 'Катар', 'Саудовская Аравия', 
                         'Афганистан', 'ОАЭ', 'Ирак', 'Индонезия', 'Япония', 'Мальдивы', 'КНР', 'Вьетнам', 'Ливан', 
                         'Непал', 'Пакистан', 'Кувейт', 'Китай'],
                'США': ['Канада', 'Аргентина', 'Бразилия', 'Мексика', 'Ямайка', 'Чили', 'Эквадор', 'Уругвай', 'Америка',
                            'Венесуэла', 'Куба', 'Коста-Рика', 'Сент-Китс и Невис'],
                'Неизвестно': ['Африка', 'Нигерия', 'Маврикий', 'Гана', 'Танзания', 'ЮАР', 'Марокко'],             
                'СНГ': ['Украина', 'Белоруссия', 'Казахстан', 'Армения', 'Узбекистан', 'Киргизия', 
                        'Азербайджан', 'Молдавия', 'Таджикистан', 'Туркменистан']}

def replace_country(value):
    """
    value: str - текущее значение страны
    return value: str - новое значение страны
    """
    for key, values in country_dict.items():
        if value in values:
            return key
    return value  # возвращаем оригинальное значение, если не найдено соответствие

# применим функцию
df['ipcountry'] = df['ipcountry'].apply(replace_country)

In [16]:
# возьмем страну как самую частую
main_country = df.groupby('donor_id')['ipcountry'].agg(lambda x: x.mode().iloc[0]).reset_index()
donor_df = donor_df.merge(main_country, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', axis=1, inplace=True)

# возьмем страну как самую частую
main_region = df.groupby('donor_id')['ipregion'].agg(lambda x: x.mode().iloc[0]).reset_index()
donor_df = donor_df.merge(main_region, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', axis=1, inplace=True)

In [17]:
# после обучения моделей стало понятно, что будет полезно 
# объединить значения стран и регионов в один признак, заполнив его следующим образом
# если страна Россия, то заполнить регионом
# если страна не Россия, то заполнить страной

donor_df.loc[donor_df.ipcountry == 'Россия', 'ipcountry'] = donor_df.loc[donor_df.ipcountry == 'Россия', 'ipregion']
donor_df.loc[donor_df.ipcountry != 'Россия', 'ipregion'] = donor_df.loc[donor_df.ipcountry != 'Россия', 'ipcountry']

# дропнем регион, так как теперь в нем нет необходимости
donor_df.drop('ipregion', inplace=True, axis=1)

# после такой замены количество уникальных регионов стало очень большим
# найдем все регионы с соответствующей им частотой
temp = donor_df['ipcountry'].value_counts().reset_index()

# все регионы, которые встретились от 50 до 500 раз объединим в группу Другое
rare_countries = list(temp[(temp['count'] > 50) & (temp['count'] <= 500)]['ipcountry'].values)

# все регионы, которые встретились менее 50 раз объединим в группу Редкие
very_rare_countries = list(temp[temp['count'] <= 50]['ipcountry'].values)

# заполним признак страны
donor_df.loc[donor_df['ipcountry'].isin(rare_countries), 'ipcountry'] = 'Другое'
donor_df.loc[donor_df['ipcountry'].isin(very_rare_countries), 'ipcountry'] = 'Редкие'

In [18]:
donor_df['ipcountry'].value_counts().reset_index()

,ipcountry,count
0,Москва,50871
1,Другое,15915
2,Санкт-Петербург,14997
3,Неизвестно,11731
4,Россия,10008
...,...,...
73,Орловская обл.,636
74,Хакасия,613
75,Азия,580
76,Ленинградская обл.,573


In [19]:
# создадим признак наличия recurring_id у донора
ids_with_rec = df[df['recurring_id']!=""]['donor_id'].unique()
donor_df['recurring'] = donor_df['id'].isin(ids_with_rec).astype('int')

# подсчитаем для доноров общую сумму всех пожертвований
temp_total_amount = df[df.status==3].groupby('donor_id')['amount_total'].sum().reset_index()

# добавим общую сумму к датасету доноров
donor_df = donor_df.merge(temp_total_amount, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', inplace=True, axis=1)

# подсчитаем для доноров среднюю сумму пожертвования
temp_mean_amount = df[df.status==3].groupby('donor_id')['amount_total'].mean().reset_index()
temp_mean_amount.rename(columns={'amount_total': 'amount_mean'}, inplace=True)

# добавим общую сумму к датасету доноров
donor_df = donor_df.merge(temp_mean_amount, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', inplace=True, axis=1)

# подсчитаем для каждого донора количество пожертвований
temp_count_of_3_status = df[df.status==3].groupby('donor_id')['status'].value_counts().reset_index()
temp_count_of_3_status.rename(columns={'count': 'count_of_3_status'}, inplace=True)

# добавим количество пожертвований к датасету доноров
donor_df = donor_df.merge(temp_count_of_3_status, how='left', left_on='id', right_on='donor_id')
donor_df.drop(['donor_id', 'status'], inplace=True, axis=1)

# подсчитаем для каждого донора количество возвратов
temp_count_of_4_status = df[df.status==4].groupby('donor_id')['status'].value_counts().reset_index()
temp_count_of_4_status.rename(columns={'count': 'count_of_4_status'}, inplace=True)

# добавим количество возвратов к датасету доноров
donor_df = donor_df.merge(temp_count_of_4_status, how='left', left_on='id', right_on='donor_id')
donor_df.drop(['donor_id', 'status'], inplace=True, axis=1)

# подсчитаем для каждого донора количество транзакций в статусе отклонен

temp_count_of_5_status = df[df.status==5].groupby('donor_id')['status'].value_counts().reset_index()
temp_count_of_5_status.rename(columns={'count': 'count_of_5_status'}, inplace=True)

# добавим количество отклоненных операций к датасету доноров
donor_df = donor_df.merge(temp_count_of_5_status, how='left', left_on='id', right_on='donor_id')
donor_df.drop(['donor_id', 'status'], inplace=True, axis=1)

# оценим как долго донор использует автоплатеж (разница между первой и последней транзакцией с recurring_id)
temp_recurring = df.groupby(['donor_id', 'recurring_id'])['when'].agg(['min', 'max']).reset_index()
temp_recurring['days_with_recurring'] = (temp_recurring['max'] - temp_recurring['min']).dt.days
temp_recurring.drop(['recurring_id', 'min', 'max'], axis=1, inplace=True)

# добавим этот признак в таблицу доноров
donor_df = donor_df.merge(temp_recurring, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', inplace=True, axis=1)

# оценим как долго донор активен (разница между первым и последним пожертвованием)
temp_active = df[df.status==3].groupby('donor_id')['when'].agg(['min', 'max']).reset_index()
temp_active['activity_time'] = (temp_active['max'] - temp_active['min']).dt.days
temp_active.drop(['min', 'max'], axis=1, inplace=True)

# добавим этот признак в таблицу доноров
donor_df = donor_df.merge(temp_active, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', inplace=True, axis=1)

# для тех, кто не жертвовал заполним пропуски
donor_df['amount_mean'].fillna(0, inplace=True)
donor_df['amount_total'].fillna(0, inplace=True)
donor_df['count_of_3_status'].fillna(0, inplace=True)
donor_df['count_of_4_status'].fillna(0, inplace=True)
donor_df['count_of_5_status'].fillna(0, inplace=True)
donor_df['days_with_recurring'].fillna(0, inplace=True)
donor_df['activity_time'].fillna(0, inplace=True)

In [20]:
# получим самый частый фонд для каждого донора
main_fund_id = df.groupby('donor_id')['fund_id'].agg(lambda x: x.mode().iloc[0]).reset_index()

# добавим его к таблице
donor_df = donor_df.merge(main_fund_id, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', axis=1, inplace=True)
              
# пропущенным значениям присовим самый популярный фонд
            
donor_df.loc[donor_df['fund_id']=="", 'fund_id'] = 'e3fd213a01aee164f475152137a83eec'

# создадим для донора метку о том, увеличил он пожертвование или нет
def get_increase(group):
    """
    value: group - группировка по донору
    return value: int - если сумма последнего пожертвования больше суммы первого пожертвования
    """
    if len(group) < 2: # проверяем жертвовал ли донор больше одного раза
        return 0
    else:
        first_donation = group['amount_total'].iloc[0]
        last_donation = group['amount_total'].iloc[-1]
        if last_donation > first_donation:
            return 1
        else:
            return 0

# сортируем по дате, чтобы правильно определить первую и последнюю пожертвования

# применяем функцию к каждой группе доноров и сбрасываем индекс
temp_increase = df.sort_values(by=['donor_id', 'when']).groupby('donor_id').apply(get_increase).reset_index(name='increase')
              
# добавим метку к таблице
donor_df = donor_df.merge(temp_increase, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', axis=1, inplace=True)

In [21]:
# создаем функцию для вычисления интервала пожертвования донора
def calculate_interval(group):
    """
    value: group - группировка по донору
    return value: int - если жертвовал более одного раза то количество дней между последним и предпоследним пожертвованием
    """
        
    if len(group) > 1:
        # находим даты последних двух пожертвований
        last_donation = group['when'].iloc[-1]
        second_last_donation = group['when'].iloc[-2]
    
        # считаем разницу в днях между последним и предпоследним
        days_between = (last_donation - second_last_donation).days
        
        return days_between
    return None

# применяем функцию к каждой группе доноров
temp_interval = df[df.status==3].sort_values(by='when').groupby('donor_id').apply(calculate_interval).reset_index(name='interval')

# добавим интервал к таблице
donor_df = donor_df.merge(temp_interval, how='left', left_on='id', right_on='donor_id')
donor_df.drop('donor_id', axis=1, inplace=True)

donor_df['interval'].fillna(temp_interval['interval'].max(), inplace=True)

# на всякий случай избавимся от возможных дубликатов
donor_df = donor_df.drop_duplicates(subset=['id'])

donor_df.drop('last_date', inplace=True, axis=1)
donor_df.head()

In [52]:
donor_df.set_index('id', inplace=True)

In [36]:
# сразу закодируем категориальные признаки

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [54]:
donor_df.head()

,only_returned_donation,days_since_last,active,gateway,multiple_country,russia,ipcountry,recurring,amount_total,amount_mean,count_of_3_status,count_of_4_status,count_of_5_status,days_with_recurring,activity_time,fund_id,increase,interval
id,,,,,,,,,,,,,,,,,,
18dd5d0a449d7db07c2a9ec0e896b193,0,1400.0,0,cloudpayments,0,1,Новосибирская область,1,21500.0,100.000000,215.0,0.0,0.0,124,124.0,030411b7e76338a91180dec47a354182,0,0.0
5c0311678207b949070c370d1fd956d3,0,23.0,1,cloudpayments,0,1,Москва,1,52000.0,881.355932,59.0,0.0,0.0,1330,2023.0,e3fd213a01aee164f475152137a83eec,1,0.0
966a63f7d5cdd1a8e203cf3325b979b8,0,4.0,1,mixplat,0,1,Астраханская обл.,0,280.0,280.000000,1.0,0.0,1.0,0,0.0,e3fd213a01aee164f475152137a83eec,0,3359.0
77cae201281c82605bb07cecbb5d80b8,0,123.0,1,cloudpayments,0,1,Россия,0,3400.0,425.000000,8.0,0.0,0.0,250,250.0,9161f2362b847ca9307ca394431ead09,0,76.0
538265b8bf808f393909bb38cb9581ec,0,784.0,0,mixplat,0,1,Татарстан,1,100.0,100.000000,1.0,0.0,1.0,0,0.0,497b5be5c4c5bf0e9a6c3791c01476f5,0,3359.0


In [56]:
# ipregion закодируем с помощью OrdinalEncoder, будем считать, что каждой территории мы присовим свой определнный код
# аналогично fund_id закодируем с помощью OrdinalEncoder, получив для каждого фонда свой номер

encoder = OrdinalEncoder()
categorical_columns = ['fund_id', 'ipcountry']
donor_df[categorical_columns] = encoder.fit_transform(donor_df[categorical_columns])

C:\Users\Huawei\AppData\Local\Temp\ipykernel_9676\1570624137.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donor_df[categorical_columns] = encoder.fit_transform(donor_df[categorical_columns])


In [57]:
# gateway будем кодировать с помощью OneHotEncoder

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first')
one_hot_encoded = encoder.fit_transform(donor_df[['gateway']]).toarray()
one_hot_columns = encoder.get_feature_names_out(['gateway'])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_columns, index=donor_df.index)

donor_df.drop('gateway', axis=1, inplace=True)
donor_df = pd.concat([donor_df, one_hot_df], axis=1)
donor_df

C:\Users\Huawei\AppData\Local\Temp\ipykernel_9676\4168297490.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donor_df.drop('gateway', axis=1, inplace=True)


,only_returned_donation,days_since_last,active,multiple_country,russia,ipcountry,recurring,amount_total,amount_mean,count_of_3_status,...,gateway_YandexMerchantOcean,gateway_cloudpayments,gateway_mir,gateway_mixplat,gateway_other,gateway_paypal,gateway_robokassa,gateway_sberbank,gateway_tinkoff,gateway_yandex
id,,,,,,,,,,,,,,,,,,,,,
18dd5d0a449d7db07c2a9ec0e896b193,0,1400.0,0,0,1,41.0,1,21500.0,100.000000,215.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5c0311678207b949070c370d1fd956d3,0,23.0,1,0,1,33.0,1,52000.0,881.355932,59.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
966a63f7d5cdd1a8e203cf3325b979b8,0,4.0,1,0,1,5.0,0,280.0,280.000000,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
77cae201281c82605bb07cecbb5d80b8,0,123.0,1,0,1,50.0,0,3400.0,425.000000,8.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538265b8bf808f393909bb38cb9581ec,0,784.0,0,0,1,65.0,1,100.0,100.000000,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74d72c53cbe6740835c86bdc957a6e04,0,569.0,0,0,1,33.0,1,500.0,500.000000,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3853faa632219fb4096c49e3246778b1,0,100.0,1,0,1,44.0,1,170.0,170.000000,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
49aeda4176a3221b92a0272a2d3422f8,0,3414.0,0,0,1,74.0,1,0.0,0.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
donor_df.to_csv("data.csv", index=False) 